# Exploratory Data Analysis: Car Sales Dataset

This notebook explores a dataset of used car listings in the U.S.  
The goal is to understand trends in pricing, car types, and other features that affect vehicle value.

In [25]:
import pandas as pd

df = pd.read_csv('../vehicles_us.csv')
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


## Duplicate Check

We checked the dataset for duplicate rows. Duplicate entries can distort our analysis, especially for price trends and vehicle type distributions. If any were found, we removed them to ensure data quality.

In [16]:
# Check how many duplicate rows are in the dataset
duplicate_rows = df.duplicated().sum()

# Print the number of duplicates found
print(f"Number of duplicate rows: {duplicate_rows}")

Number of duplicate rows: 0


In [17]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,47906.000000,46265.000000,43633.000000,25572.0,51525.00000
mean,12132.464920,2009.750470,6.125235,115553.461738,1.0,39.55476
std,10040.803015,6.282065,1.660360,65094.611341,0.0,28.20427
min,1.000000,1908.000000,3.000000,0.000000,1.0,0.00000
25%,5000.000000,2006.000000,4.000000,70000.000000,1.0,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,1.0,33.00000
75%,16839.000000,2014.000000,8.000000,155000.000000,1.0,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.0,271.00000


In [18]:
df.isna().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

## Handling Missing Values
We used different strategies to fill in missing data based on the type of information. For example:
- Replaced missing `is_4wd` with 0 (assumes not 4WD).
- Replaced missing `paint_color` with 'Unknown'.
- Filled `model_year`, `odometer`, and `cylinders` using the median value grouped by model.

In [ ]:
# Fill missing is_4wd values with 0 (assume not 4WD)
df['is_4wd'] = df['is_4wd'].fillna(0)

# Fill missing paint_color with 'Unknown'
df['paint_color'] = df['paint_color'].fillna('Unknown')

# Fill model_year using median of each model group
df['model_year'] = df['model_year'].fillna(df.groupby('model')['model_year'].transform('median'))

# Fill odometer using median of each model group
df['odometer'] = df['odometer'].fillna(df.groupby('model')['odometer'].transform('median'))

# Fill cylinders using median of each model group
df['cylinders'] = df['cylinders'].fillna(df.groupby('model')['cylinders'].transform('median'))

print(df.isna().sum()) # See how many missing values are left 

price            0
model_year       0
model            0
condition        0
cylinders        0
fuel             0
odometer        41
transmission     0
type             0
paint_color      0
is_4wd           0
date_posted      0
days_listed      0
dtype: int64


In [20]:
import plotly.express as px

In [21]:
# Creating a histogram to explore how car prices are distributed across different vehicle types
# The x-axis shows the price of vehicles
# The y-axis shows how many vehicles fall into each price range (count)
# We use color='type' so each vehicle type is represented with a different color

# This helps us compare price distributions for different types of cars like SUVs, sedans, trucks, etc.
# It also shows which vehicle types are most common and how their prices are spread out

# Histogram of prices by vehicle type
px.histogram(df, x='price', color='type', nbins=50, title='Price Distributation by Vehicle Type')

Histogram: Price Distribution by Vehicle Type

This chart shows that most vehicle types are priced under $50,000. SUVs and sedans are the most common in the dataset. Expensive cars are rare and mostly belong to specific types like pickup and convertible.


In [22]:
# Creating a scatter plot to explore the relationship between car price and odometer reading
# The x-axis shows the mileage (odometer) of each vehicle
# The y-axis shows the price of the vehicle
# Each dot represents one car, and the color shows its condition (e.g., new, fair, salvage, etc.)

# This chart helps us understand how car condition and mileage affect pricing
# For example, we can see that lower mileage cars tend to be more expensive, especially when they are in better condition

# Scatterplot: Price vs. Odometer, colored by condition
px.scatter(df, x='odometer', y='price', color='condition', title='Price vs. Odometer by Condition')

Scatter Plot: Price vs. Odometer by Condition

Cars with lower mileage tend to be more expensive. The best condition vehicles (“new”, “like new”, and “excellent”) generally have lower odometer readings and higher prices.

In [23]:
# Creating a box plot to compare the price ranges of cars based on transmission type
# The x-axis shows the type of transmission: automatic, manual, or other
# The y-axis shows the price of the vehicles

# Box plots show the median, quartiles, and any outliers
# This helps us see how prices vary for each transmission type and if there are any extreme price values
# It's useful for understanding whether one type tends to be more expensive or has more variation in price

# Box plot: Price range by transmission type
px.box(df, x='transmission', y='price', title='Price Range by Transmission Type')

Box Plot: Price Range by Transmission Type

Automatic cars are more common and have a wider price range. Manual and “other” transmissions tend to have lower prices overall and fewer extreme outliers.

In [24]:
# Creating a bar chart to show the average car price for each model year
# First, we group the data by 'model_year' and calculate the average price for each year
# Then, we reset the index so it's easier to plot with Plotly

# This chart helps us identify how car prices have changed over time
# It can reveal trends such as older cars being less expensive or more recent models being priced higher
# It's useful for understanding long-term pricing patterns across different model years

# Bar chart: Average price by model year
avg_price = df.groupby('model_year')['price'].mean().reset_index()
px.bar(avg_price, x='model_year', y='price', title='Average Car Price by Model Year')

Bar Chart: Average Car Price by Model Year

Newer model years tend to have higher average prices, while older cars show more variability in pricing. There are noticeable spikes and dips that may reflect unique collector models or data irregularities.

## Summary of Insights

In this analysis, I explored how car prices relate to different features using visualizations. 
Here are the key insights:

    •    __Price Distribution by Vehicle Type__: Most vehicles fall under lower price ranges, especially sedans and SUVs. 
However, some types like convertibles and trucks show a wider spread in prices, possibly due to special models or features.

    •    __Price vs. Odometer by Condition__: Cars with fewer miles on the odometer generally have higher prices. 
Also, vehicles listed as “new” or “like new” tend to be more expensive compared to those in “fair” or “salvage” condition.

    •    __Price Range by Transmission Type__: Automatic cars are the most common and have a wider price range.
Manual and “other” transmission types are fewer and often cheaper, though there are some outliers.

    •    __Average Car Price by Model Year__: Older vehicles (pre-2000) are generally cheaper, but some vintage models 
may still have high prices. Newer cars (post-2010) show a clear increase in average price, which is expected.